#Flattening Tracks 

First, we'll get the list of files from the ./midi/pop directory.

In [13]:
import os
import midi
import fluidsynth

In [14]:
midi_path = '../midi/pop/'
midi_files = os.listdir(midi_path)

In [15]:
len(midi_files)

8233

Success. Now, we want to figure out which instruments are on which channels for each file. The following function returns a dictionary with keys 0-15 representing the channels and values representing a list of instruments assigned to that channel at various points of the composition.

In [17]:
def get_channel_mappings(mfile):
    """Return a dictionary of which instruments are used on each of the 16 channels.
    
    Input: midi.containers.Pattern object
    Output: dictionary
    """
    
    channel_map = {i:None for i in range(16)}
    for track in mfile:
        for event in track:
            if isinstance(event, midi.ProgramChangeEvent):
                if channel_map[event.channel] is None:
                    channel_map[event.channel] = event.data
                elif event.data[0] in channel_map[event.channel]:
                    continue
                else:
                    channel_map[event.channel] += event.data
    return channel_map

So now, we just find all the mappings for our corpus of songs.

In [18]:
all_channel_mappings = []

for i in range(len(midi_files)):
    try:
        mfile = midi.read_midifile(midi_path+midi_files[i])
        mfile.make_ticks_abs()
        all_channel_mappings.append(get_channel_mappings(mfile))
    except:
        print "Problem reading " + midi_files[i]

Nothing was lost! Phew.

In [19]:
len(all_channel_mappings)

8233

Let's save this for later.

In [20]:
import pickle
with open('../channel_mappings.pkl', 'w') as f:
    pickle.dump(all_channel_mappings, f)